This notebook generates the values reported in the text of
`Systematic over-crediting in California’s forest carbon offsets program`


In [1]:
from collections import Counter
import fsspec
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error

from carbonplan_forest_offsets.analysis.project_crediting_error import get_slag_to_total_scalar
from carbonplan_forest_offsets.data import cat
from carbonplan_forest_offsets.load.issuance import load_issuance_table, ifm_opr_ids

In [2]:
df = load_issuance_table(forest_only=False)
retro_json = cat.retro_db_light_json.read()

## Total Value


In [3]:
total_arbocs = df["allocation"].sum()
display(total_arbocs)

price_per_arboc = 13.67

display(price_per_arboc * total_arbocs)

192996530.0

2638262565.1

### figure 1a

data also posted to results via `analysis/summarize_issuance.py


In [4]:
def subset_stats(criteria):
    nunique_opr_ids = df[criteria]["opr_id"].nunique()
    total_arbocs = (df[criteria]["allocation"].sum() / 1_000_000).round(1)
    return (nunique_opr_ids, total_arbocs)


upfront_opr_ids = [
    project["opr_id"]
    for project in retro_json
    if project["carbon"]["initial_carbon_stock"]["value"]
    > project["carbon"]["common_practice"]["value"]
]
compliance_opr_ids = [project["opr_id"] for project in retro_json]

subsets = {
    "all": np.tile(True, len(df)),
    "all_forest": df["project_type"] == "forest",
    "compliance_ifm": (df["opr_id"].isin(ifm_opr_ids)) & (df["Early Action/ Compliance"] == "COP"),
    "non_graduated_compliance_ifms": (df["opr_id"].isin(compliance_opr_ids))
    & (df["Early Action/ Compliance"] == "COP"),
    "upfront_ifm": (df["opr_id"].isin(upfront_opr_ids)) & (df["arb_rp_id"].isin(["A"])),
}

summary = {k: subset_stats(v) for k, v in subsets.items()}
display(summary)

{'all': (474, 193.0),
 'all_forest': (130, 156.1),
 'compliance_ifm': (93, 137.1),
 'non_graduated_compliance_ifms': (74, 131.5),
 'upfront_ifm': (72, 121.0)}

In [5]:
summary["upfront_ifm"][1] / summary["all_forest"][1]

0.7751441383728379

In [6]:
summary["upfront_ifm"][1] / summary["all"][1]

0.6269430051813472

## Carbon stocks for SC by species


In [7]:
sc_data = cat.rfia_all(assessment_area_id=297).read()

sc_data = sc_data[sc_data["YEAR"] == 2010].copy()  # use 2010 because comporable to CP data


sc_data["CARB_ACRE"] = sc_data["CARB_ACRE"] * 44 / 12 * 0.907185

In [8]:
fortyps_of_interest = {"Douglas fir": 201, "Ponderosa pine": 221, "Tanoak": 941}

In [9]:
standing_carbon = {}
for k, v in fortyps_of_interest.items():
    standing_carbon[k] = round(sc_data.loc[sc_data["FORTYPCD"] == v, "CARB_ACRE"].item(), 1)
display(standing_carbon)

{'Douglas fir': 122.5, 'Ponderosa pine': 60.4, 'Tanoak': 192.4}

### Coastal Alaska


In [10]:
# ak has three assessment areas but lets summarize across all to report inline value
ak_assessment_areas = [285, 286, 287]
ak_all = pd.concat([cat.rfia_all(assessment_area_id=aa_id).read() for aa_id in ak_assessment_areas])

ak_all = ak_all[ak_all["YEAR"] == 2013].copy()  # 2013 to match what used in CP

sums = ak_all.groupby("FORTYPCD").sum()[["CARB_TOTAL", "AREA_TOTAL", "nPlots_TREE"]]

In [11]:
ak_forests_of_interest = {
    "Sitka sprice": 305,
    "Western hemlock": 301,
    "Cottonwood": 703,
    "Paper birch": 902,
}

In [12]:
ak_carb_acre = sums["CARB_TOTAL"] / sums["AREA_TOTAL"] * 44 / 12 * 0.907185

display({k: ak_carb_acre[v].round(1) for k, v in ak_forests_of_interest.items()})

{'Sitka sprice': 121.1,
 'Western hemlock': 143.0,
 'Cottonwood': 41.4,
 'Paper birch': 38.3}

### Project Examples


In [13]:
with fsspec.open(
    "https://carbonplan.blob.core.windows.net/carbonplan-retro/results/reclassification-crediting-error.json",
    mode="r",
) as f:
    reclass_credit = json.load(f)

In [14]:
def summarize_project(project):
    crediting_error = np.percentile(
        reclass_credit[project["opr_id"]]["delta_arbocs"], q=[5, 50, 95]
    ).round(1)

    as_frac = crediting_error / project["arbocs"]["calculated"]
    print(f"{project['opr_id']} has a {crediting_error[1]} crediting error ({as_frac[1].round(3)})")
    print(f"CI: {crediting_error[0]}, {crediting_error[2]}")
    print(f"% CI: {as_frac[0].round(3)}, {as_frac[2].round(3)}")

In [15]:
opr_id = "ACR189"
project = [x for x in retro_json if x["opr_id"] == opr_id][0]

summarize_project(project)

ACR189 has a 135869.2 crediting error (0.501)
CI: 85480.8, 185916.6
% CI: 0.315, 0.686


In [16]:
opr_id = "ACR361"
project = [x for x in retro_json if x["opr_id"] == opr_id][0]

summarize_project(project)

ACR361 has a 318269.7 crediting error (0.134)
CI: -198607.4, 871385.6
% CI: -0.084, 0.367


In [17]:
opr_id = "CAR1183"
project = [x for x in retro_json if x["opr_id"] == opr_id][0]

summarize_project(project)

CAR1183 has a 4417068.0 crediting error (1.0)
CI: 4417068.0, 4417068.0
% CI: 1.0, 1.0


### Ecosection combos


In [18]:
fn = f"https://carbonplan.blob.core.windows.net/carbonplan-retro/results/297_by_ecosection.csv"
with fsspec.open(fn, mode="r") as f:
    data = pd.read_csv(f)

data = data[data["YEAR"] == 2010]  # match CP calculation period

In [19]:
data["ECOSECTION"] = data["ECOSUBCD"].str[:-1].str.strip()


lst = ["M261A", "M261B", "M261D"]  # ['263A']# ["M221A","M221B","M221C"] #['263A']#
subset = data[(data["CARB_TOTAL"] > 0) & (data["ECOSECTION"].isin(lst))].copy()

In [20]:
sums = subset.groupby(["site", "ECOSECTION"])[["CARB_TOTAL", "AREA_TOTAL"]].sum()
slag = sums["CARB_TOTAL"] / sums["AREA_TOTAL"] * 44 / 12 * 0.907185

In [21]:
ss_average = subset.groupby(["site"])[["CARB_TOTAL", "AREA_TOTAL"]].sum()

In [22]:
ss_slag = ss_average["CARB_TOTAL"] / ss_average["AREA_TOTAL"] * 44 / 12 * 0.907185

In [23]:
slag.unstack(1).round(1)

ECOSECTION,M261A,M261B,M261D
site,,,
high,120.6,150.5,100.6
low,95.2,106.4,53.2


In [24]:
((ss_slag - slag) / slag).unstack(0)

site,high,low
ECOSECTION,,
M261A,-0.013284,-0.161891
M261B,-0.209140,-0.250049
M261D,0.183504,0.499957


In [25]:
(121.8 - 150.5) / 150.5

-0.19069767441860466

# Methods


## Classifier


In [26]:
fn = f"https://carbonplan.blob.core.windows.net/carbonplan-retro/reclassification/classifier_fscores.json"
with fsspec.open(fn, mode="r") as f:
    fscores = json.load(f)

In [27]:
np.median([scores[0] for ss_name, scores in fscores.items()])

0.77988012574225

In [28]:
table = (
    pd.Series({ss_name: round(scores[0], 2) for ss_name, scores in fscores.items()})
    .sort_values(ascending=False)
    .rename("Weighted F-Score")
    .to_frame()
)

In [29]:
table.index = table.index.set_names("Superseciton")

In [30]:
table

,Weighted F-Score
Superseciton,
Central New Mexico,0.94
White Mountains - San Francisco Peaks - Mongollon,0.93
Southeast and South Central Alaska,0.91
Columbia Basin,0.89
Okanogan Highland,0.88
Northwest Cascades,0.87
Southern Cascades,0.82
Laurentian Mixed Forest Southern Superior,0.79
White Mountains,0.78


## Common Practice Stuff


In [31]:
projects = [
    x
    for x in retro_json
    if x["carbon"]["initial_carbon_stock"]["value"] > x["carbon"]["common_practice"]["value"]
]

In [32]:
cp_df = pd.DataFrame(
    [
        (
            project["carbon"]["average_slag_baseline"]["value"],
            project["carbon"]["common_practice"]["value"],
        )
        for project in projects
    ],
    columns=["baseline", "cp"],
)

## Figure 2 Caption

What percent of projects are within 5 percent of common practice?


In [33]:
sum((cp_df["baseline"] <= cp_df["cp"] * 1.05)) / len(cp_df.dropna())

0.8923076923076924

# Extended Methods


In [34]:
fn = f"https://carbonplan.blob.core.windows.net/carbonplan-retro/results/common-practice-verification.json"
with fsspec.open(fn, mode="r") as f:
    cp_verification = json.load(f)

In [35]:
data = pd.DataFrame(cp_verification["assessment_areas"])[["common_practice", "rfia"]]
mean_squared_error(data["common_practice"], data["rfia"]) ** 0.5

4.9433707752078595

In [36]:
data = pd.DataFrame(cp_verification["projects"])[["opr_id", "recalculated", "project_reported"]]
mean_squared_error(data["recalculated"], data["project_reported"]) ** 0.5

9.714909886835764

In [37]:
data["diff"] = (data["recalculated"] - data["project_reported"]) / (data["project_reported"])
data = data[np.isfinite(data["diff"])]  # CAR1186 = infite bc original CP = 0.

In [38]:
data["diff"].mean()

0.02872469944696075

In [39]:
error_cp0 = data.set_index("opr_id")["diff"]

KeyError: "None of ['opr_id'] are in the columns"

## Project scalars

ratio of above ground to below ground carbon


In [ ]:
pd.DataFrame([get_slag_to_total_scalar(project) for project in projects]).describe()

## IFM-3

IFM-3 in the baseline scenario and project scenario are equal


In [ ]:
sum([x["rp_1"]["ifm_3"] == x["baseline"]["ifm_3"] for x in projects]) / len(projects)

In [ ]:
fn = "https://carbonplan.blob.core.windows.net/carbonplan-retro/results/reclassification-crediting-error.json"
with fsspec.open(fn, mode="r") as f:
    crediting_error = json.load(f)

crediting_df = pd.DataFrame({k: v["delta_arbocs"] for k, v in crediting_error.items()})

median_crediting_error = {k: np.median(v["delta_arbocs"]) for k, v in crediting_error.items()}

In [ ]:
tp = pd.concat(
    [pd.Series(median_crediting_error).rename("crediting_error"), error_cp0.rename("cp")],
    axis=1,
)

In [ ]:
tp.dropna().corr()

In [ ]:
sns.jointplot("cp", "crediting_error", data=tp)

# Disucssion

## Data limitations


### Plots in AK


In [ ]:
ak_assessment_areas = [285, 286, 287]
ak_all = pd.concat([cat.rfia_all(assessment_area_id=aa_id).read() for aa_id in ak_assessment_areas])

In [ ]:
ak_all = ak_all[ak_all["YEAR"] == 2013]

In [ ]:
plots_by_aa = ak_all.groupby("Assessment")["nPlots_TREE"].sum()
display(plots_by_aa)

In [ ]:
sum([project["arbocs"]["issuance"] for project in retro_json if 287 in project["supersection_ids"]])

In [ ]:
# total AK plots
display(plots_by_aa.sum())

Contrasted against 297 (mixed conifer)


In [ ]:
sc = cat.rfia_all(assessment_area_id=297).read()
sc[sc["YEAR"] == 2010].nPlots_TREE.sum()